In [ ]:
import os
import pandas as pd

from onekey_algo import OnekeyDS as okds
from onekey_algo import get_param_in_cwd

data_pattern = get_param_in_cwd('data_pattern', os.path.join(okds.ct, 'crop')) 
patches = []
for r, ds, fs in os.walk(data_pattern):
    patches.extend([os.path.join(r, p) for p in fs if p.endswith('.jpg')])

patches = pd.DataFrame(patches, columns=['fpath'])
patches['fname'] = patches['fpath'].map(lambda x: os.path.basename(x))

def map2id(x):
    x = os.path.basename(x)
    *fname_items, w, h, ltx, lty, rbx, rby = x.split('_')
    return '_'.join(fname_items)
patches['ID'] = patches['fpath'].map(map2id)
patches

In [ ]:
label_f = pd.read_csv(get_param_in_cwd('label_file'))
label_f['ID'] = label_f['ID'].map(lambda x: str(x))
patches = pd.merge(patches, label_f, on='ID', how='inner')
patches

In [ ]:
import numpy as np

ugroups = np.unique(patches['group'])
for group in ugroups:
    group_info = 'train' if group == 'train' else 'val'
    patches[patches['group'] == group][['fpath', 'label']].to_csv(os.path.join('split_info', f'{group_info}.txt'), 
                                                                  index=False, header=False, sep='\t')